In [19]:
import pandas as pd
import jieba.posseg as pseg
import keras

train = pd.read_csv('/home/au/文件/Data/NLP/train.csv', encoding = 'utf-8')
train = train.astype(str)
cols = ['title1_zh', 'title2_zh', 'label']
train = train.loc[0:100, cols]
# train.head(3)

def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([
        word for word, flag in words if flag != 'x'])

train['title1_tokenized'] = \
    train.loc[0:, 'title1_zh'] \
         .apply(jieba_tokenizer)
train['title2_tokenized'] = \
    train.loc[0:, 'title2_zh'] \
         .apply(jieba_tokenizer)

train.iloc[0:, [0, 3]].head()
train.iloc[0:, [1, 4]].head()

,title2_zh,title2_tokenized
0,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,警方 辟谣 鸟巢 大会 每人 领 5 万 仍 有 老人 坚持 进京
1,深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,深圳 GDP 首 超 香港 深圳 统计局 辟谣 只是 差距 在 缩小
2,GDP首超香港？深圳澄清：还差一点点……,GDP 首 超 香港 深圳 澄清 还 差 一点点
3,去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,去年 深圳 GDP 首 超 香港 深圳 统计局 辟谣 还 差 611 亿
4,吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,吃 了 30 年 食用油 才 知道 一片 大蒜 轻松 鉴别 地沟油


maxword = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=maxword)

In [8]:
corpus_x1 = train.title1_tokenized
corpus_x2 = train.title2_tokenized
corpus = pd.concat([
    corpus_x1, corpus_x2])
corpus.shape
print(corpus)

0               2017 养老保险 又 新增 两项 农村 老人 人人 可 申领 你 领到 了 吗
1        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
2        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
3        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
4                             用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
5        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
6                                      吃 榴莲 的 禁忌 吃 错会 致命
7        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
8              旅行 青蛙 居然 是 一款 生育 意愿 测试 器 大家 还是 玩 珠宝 V 课 吧
9                             用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
10                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
11                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
12                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
13                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
14             飞机 就要 起飞 一个 男人 在 机舱 口 跪下 这 是 今天 最 催泪 的 一幕
15                飞机 就要 起飞 一个 男人 在 机舱 口 跪下 这 是 最 催泪 的 一幕
16                           男人 在 机舱 口 跪下 原来 一切 都 只 因为 爱
17             飞机 就要 起飞 一个 男人 在

In [9]:
MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)
x1_train = tokenizer \
    .texts_to_sequences(corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(corpus_x2)
len(x1_train)
x1_train[:1]

[[318, 319, 234, 320, 321, 322, 235, 323, 184, 324, 9, 325, 12, 27]]

In [10]:
for seq in x1_train[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['2017', '养老保险', '又', '新增', '两项', '农村', '老人', '人人', '可', '申领', '你', '领到', '了', '吗']


In [11]:
for seq in x1_train[:10]:
    print(len(seq), seq[:5], ' ...')

14 [318, 319, 234, 320, 321]  ...
19 [9, 78, 59, 29, 120]  ...
19 [9, 78, 59, 29, 120]  ...
19 [9, 78, 59, 29, 120]  ...
9 [55, 52, 28, 25, 1]  ...
19 [9, 78, 59, 29, 120]  ...
7 [2, 236, 1, 124, 2]  ...
19 [9, 78, 59, 29, 120]  ...
16 [97, 328, 237, 64, 329]  ...
9 [55, 52, 28, 25, 1]  ...


In [12]:
max_seq_len = max([
    len(seq) for seq in x1_train])
max_seq_len

22

In [13]:
MAX_SEQUENCE_LENGTH = 20
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
for seq in x1_train + x2_train:
    assert len(seq) == 20
    
print("所有新聞標題的序列長度皆為 20 !")

所有新聞標題的序列長度皆為 20 !


In [15]:
import numpy as np 

# 定義每一個分類對應到的索引數字
label_to_index = {
    'unrelated': 0, 
    'agreed': 1, 
    'disagreed': 2
}

# 將分類標籤對應到剛定義的數字
y_train = train.label.apply(
    lambda x: label_to_index[x])

y_train = np.asarray(y_train) \
            .astype('float32')

y_train[:5]

array([0., 0., 0., 0., 1.], dtype=float32)

In [16]:
y_train = keras \
    .utils \
    .to_categorical(y_train)

y_train[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [17]:
from sklearn.model_selection \
    import train_test_split

VALIDATION_RATIO = 0.1
# 小彩蛋
RANDOM_STATE = 9527

x1_train, x1_val, \
x2_train, x2_val, \
y_train, y_val = \
    train_test_split(
        x1_train, x2_train, y_train, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [18]:
from keras import layers
rnn = layers.SimpleRNN()

TypeError: __init__() missing 1 required positional argument: 'units'

In [ ]:
state_t = 0
# 細胞 A 會重複執行以下處理
for input_t in input_sequence:
    output_t = f(input_t, state_t)
    state_t = output_t

In [ ]:
from keras import layers
lstm = layers.LSTM()

In [ ]:
for i, seq in enumerate(x1_train[:5]):
    print(f"新聞標題 {i + 1}: ")
    print(seq)
    print()

In [ ]:
x1_train.shape

In [ ]:
from keras import layers
embedding_layer = layers.Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

In [ ]:
# 基本參數設置，有幾個分類
NUM_CLASSES = 3

# 在語料庫裡有多少詞彙
MAX_NUM_WORDS = 10000

# 一個標題最長有幾個詞彙
MAX_SEQUENCE_LENGTH = 20

# 一個詞向量的維度
NUM_EMBEDDING_DIM = 256

# LSTM 輸出的向量維度
NUM_LSTM_UNITS = 128

# 建立孿生 LSTM 架構（Siamese LSTM）
from keras import Input
from keras.layers import Embedding, \
    LSTM, concatenate, Dense
from keras.models import Model

# 分別定義 2 個新聞標題 A & B 為模型輸入
# 兩個標題都是一個長度為 20 的數字序列
top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')

# 詞嵌入層
# 經過詞嵌入層的轉換，兩個新聞標題都變成
# 一個詞向量的序列，而每個詞向量的維度
# 為 256
embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

# LSTM 層
# 兩個新聞標題經過此層後
# 為一個 128 維度向量
shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)

# 串接層將兩個新聞標題的結果串接單一向量
# 方便跟全連結層相連
merged = concatenate(
    [top_output, bm_output], 
    axis=-1)

# 全連接層搭配 Softmax Activation
# 可以回傳 3 個成對標題
# 屬於各類別的可能機率
dense =  Dense(
    units=NUM_CLASSES, 
    activation='softmax')
predictions = dense(merged)

# 我們的模型就是將數字序列的輸入，轉換
# 成 3 個分類的機率的所有步驟 / 層的總和
model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)

from keras.utils import plot_model
plot_model(
    model, 
    to_file='model.png', 
    show_shapes=True, 
    show_layer_names=False, 
    rankdir='LR')